In [112]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
import random as rd
import itertools
import matplotlib.pyplot as plt
import heapq
from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import MeanShift,estimate_bandwidth
from sklearn.cluster import DBSCAN   
from sklearn.neighbors import NearestNeighbors 
from sklearn.preprocessing import StandardScaler

%matplotlib inline
from sklearn.mixture import BayesianGaussianMixture,GaussianMixture
import scipy
from collections import Counter

import itertools
import time
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
import pyecharts

from allpairspy import AllPairs
import itertools
from sklearn.metrics import classification_report,accuracy_score

plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False  

# test ODD

In [113]:
ODD_FILE_PATH = './Testing_Results.csv'
odd = pd.read_csv(ODD_FILE_PATH)
odd

,vs,dif_speed,dis,result
0,1,0,5,0
1,1,0,6,0
2,1,0,7,0
3,1,0,8,0
4,1,0,9,0
...,...,...,...,...
68330,43,2,46,0
68331,43,2,47,0
68332,43,2,48,0
68333,43,2,49,0


In [114]:
FEATURE = ['vs','dif_speed','dis']
name = np.array(FEATURE)
X_qualified_All=odd[FEATURE].values
Y_all=odd['result'].values

# Calculate the PDF of each scenarios

In [115]:
DATA_FILE_PATH = './carfollowing_from_HighD.csv'
data = pd.read_csv(DATA_FILE_PATH)
data=data[['vs','dif_speed','dis']]
data

,vs,dif_speed,dis
0,35.02,0.80,22.67
1,36.62,1.71,25.39
2,24.53,-3.85,38.06
3,36.57,0.74,38.83
4,31.15,-0.16,35.29
...,...,...,...
30287,34.96,1.00,48.41
30288,31.91,-0.45,35.03
30289,32.76,-3.43,21.48
30290,32.84,-3.11,47.72


In [116]:
import warnings
warnings.filterwarnings("ignore")

rdList = np.random.choice(data.shape[0],size=5000)

x_train = np.array(data)[rdList,:]
x_f = x_train

BEST_COMPONENTS = 0
BEST_BIC = 99999
for i in range(20):
    BGMM = GaussianMixture(n_components=i+1,tol=0.00001,max_iter=10000)
    BGMM.fit(x_f)
    bic = BGMM.bic(x_f)
    if bic < BEST_BIC:
        BEST_BIC = bic
        BEST_COMPONENTS = i+1
BGMM = GaussianMixture(n_components=BEST_COMPONENTS,tol=0.0001,max_iter=10000)
BGMM.fit(x_f) 
print("best_components:%d"%BEST_COMPONENTS)

best_components:7


In [117]:
def computeGMMpdf(BGMM, sample,scaler=None):
    if scaler is not None:
        sample = scaler.transform(sample)
    for i in range(BGMM.weights_.shape[0]):
        weights = BGMM.weights_[i] 
        mean = BGMM.means_[i, :]
        cov = BGMM.covariances_[i, :, :]
        mn = scipy.stats.multivariate_normal(mean=mean, cov=cov)
        if i == 0:
            res = mn.pdf(sample) * weights
        else:
            res += mn.pdf(sample) * weights 
    return res

In [118]:
score = computeGMMpdf(BGMM,X_qualified_All)
p_x_all = score/score.sum() 

In [119]:
score.sum()

0.9905042959952041

In [120]:
p_x_all

array([3.38966684e-06, 4.19142504e-06, 5.03424736e-06, ...,
       2.12069192e-06, 1.37745479e-06, 7.54767426e-07])

In [121]:
x_p_frame = pd.DataFrame(X_qualified_All)
x_p_frame.columns = ['vs','dif_speed','dis']
x_p_frame['p'] = p_x_all
danger_rate=p_x_all[Y_all==1].sum()
danger_rate

5.149858806861421e-07

# Evaluation Indicators

In [122]:
evaluation_list = ['Samples','Methods','Input coverage','2-wise coverage','Number of collisions','Coverage of collisions','Collision rate estimation','Estimation Error'] 
evaluation_matrix = pd.DataFrame(columns=evaluation_list)

In [123]:
def evaluation(time,name,sample,result,p_is=None,ment=False):
    array_tmp = [time,name]
    global evaluation_matrix,danger_rate,x_p_frame # 定义全局变量，然后会在函数外面寻找变量的定义，在函数内部对变量操作之后，依然是全局变量。

    # Input coverage
    ODD_cov=sample.shape[0]/68335
    array_tmp+=[ODD_cov]
    
    # 2-wise coverage
    pair_all_modify = 6566
    pair_found = np.unique(sample[:,0:2],axis=0).shape[0] + np.unique(sample[:,1:3],axis=0).shape[0] + np.unique(sample[:,[0,2]],axis=0).shape[0]
    pair_cov=pair_found/pair_all_modify
    array_tmp+=[pair_cov]
    
    # Number of collisions
    danger_find=result.sum()
    array_tmp+=[danger_find]
    
    # Coverage of collisions
    danger_cov=danger_find/6320
    array_tmp+=[danger_cov]
    
    # Collision rate estimation
    sample= pd.DataFrame(sample)
    sample['Y']=result
    sample.columns = ['vs','dif_speed','dis','Y']
    sample_with_rate=pd.merge(sample,x_p_frame,how='left')
    collision_rate=sample_with_rate[sample_with_rate.Y==1].p.sum()
    array_tmp+=[collision_rate]  
    
    
    # Estimation Error
    array_tmp += [np.abs(collision_rate-danger_rate)/danger_rate]

    
    #to evaluation_matrix
    evaluation_matrix.loc[evaluation_matrix.shape[0]] = array_tmp
    return array_tmp

# PSO

In [124]:
from test_base_data import testBaseData
sut = testBaseData("Testing_Results.csv",name)

(68335, 4)


In [125]:
from Grid_Search import grid
para_tuning = grid()
print(para_tuning.shape)

(225, 3)


In [ ]:
%%time

k = 0

#from idm_pso import IDMDanger
#sut_danger=IDMDanger(name)

from pso2 import PSO
test_time=[1690,3000,4000,5000,6000,7000,8000,9000,10000,20000,30000,40000,50000,60000,68335]

from Grid_Search import grid
para_tuning = grid()
print(para_tuning.shape[0])
for i in range(para_tuning.shape[0]):
    para_1,para_2,para_3 = para_tuning[i][0],para_tuning[i][1],para_tuning[i][2]

    for NUM in test_time:
        SAMPLE_NUM = NUM
        pso = PSO(X_qualified_All,upper_bound,lower_bound,pN=int(SAMPLE_NUM*0.1),w=para_1,c1=para_2,c2=para_3)#int(SAMPLE_NUM*0.2)
        sample = pso.init_population(X_qualified_All,pN=int(SAMPLE_NUM*0.1))
        result = -sut.test(sample)
        history=sample
        for i in range(10):
            sample, fit = pso.train(sample,result)
            result = -sut.test(sample)
            history=np.concatenate((history,sample),axis=0)
        
        
        sample_use=np.unique(history,axis=0)
        result_use=sut.test(sample_use)
        result_use=np.nan_to_num(result_use, nan=0) #only count avoidable collisions
        
        temp_evaluate_result=evaluation(SAMPLE_NUM,'pso',sample_use,result_use)
        
        print(temp_evaluate_result)

225
[1690, 'pso', 0.012409453427965171, 0.18885166006701187, 106.0, 0.016772151898734176, 8.4468155600711e-12, 0.9999835979667077]
[3000, 'pso', 0.01876051803614546, 0.25434054218702407, 193, 0.030537974683544303, 7.345861376332094e-10, 0.9985735800432928]
[4000, 'pso', 0.02413111875320114, 0.2908924763935425, 191, 0.030221518987341774, 3.149875529567556e-10, 0.9993883569146846]
[5000, 'pso', 0.02885783273578693, 0.33125190374657326, 668, 0.10569620253164556, 2.2840852794111766e-10, 0.9995564761355462]
[6000, 'pso', 0.03422843345284261, 0.36323484617727686, 300, 0.04746835443037975, 1.566838567128766e-07, 0.6957511601985696]
[7000, 'pso', 0.03788688080778518, 0.38486140724946694, 287.0, 0.04541139240506329, 1.542472942130884e-10, 0.9997004824792332]
[8000, 'pso', 0.04154532816272774, 0.40572646969235454, 275, 0.043512658227848104, 3.4133141935653756e-10, 0.999337202373584]
[9000, 'pso', 0.04434038194190386, 0.42141334145598536, 280, 0.04430379746835443, 7.597245840341784e-09, 0.9852476

[7000, 'pso', 0.02413111875320114, 0.3670423393237892, 578.0, 0.09145569620253165, 4.892471440893706e-09, 0.9904997949955924]
[8000, 'pso', 0.024160386332040682, 0.36826073713067314, 630.0, 0.09968354430379747, 1.1836872424017607e-07, 0.7701515154503511]
[9000, 'pso', 0.03330650471939709, 0.4466950959488273, 645.0, 0.10205696202531646, 4.982086772447869e-09, 0.9903257798722366]
[10000, 'pso', 0.03602838955147435, 0.4791349375571124, 579.0, 0.09161392405063291, 3.3721049562997834e-09, 0.9934520438661213]
[20000, 'pso', 0.09017341040462427, 0.9000913798355163, 1422.0, 0.225, 5.577459706585792e-08, 0.8916968422677015]
[30000, 'pso', 0.12312870417794688, 1.03396283886689, 1992.0, 0.31518987341772153, 1.8112964217107624e-08, 0.9648282314206851]
[40000, 'pso', 0.13303577961513133, 1.0112701797136765, 1809.0, 0.2862341772151899, 6.944557743342967e-08, 0.8651505215232235]
[50000, 'pso', 0.12078729787078364, 1.1558026195552848, 2095.0, 0.3314873417721519, 1.20959646413141e-07, 0.765120460677523

[30000, 'pso', 0.05843272115314261, 0.6446847395674687, 604.0, 0.09556962025316455, 1.1441569207323355e-07, 0.7778275165121195]
[40000, 'pso', 0.13234799151240215, 0.8754188242461164, 1085.0, 0.17167721518987342, 2.271131781880222e-08, 0.9558991446745244]
[50000, 'pso', 0.11224116484963781, 0.7755102040816326, 1104.0, 0.17468354430379746, 6.958874901736014e-09, 0.9864872510825649]
[60000, 'pso', 0.10160239994146485, 0.9089247639354249, 565.0, 0.0893987341772152, 4.620661620144166e-08, 0.9102759552555534]


In [ ]:
evaluation_matrix

In [ ]:
data=evaluation_matrix

import math

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',  # 设置字体类型
    "font.size": 18,
#     "mathtext.fontset":'stix',
}
rcParams.update(config)

x=np.log([1690,3000,4000,5000,6000,7000,8000,9000,10000,20000,30000,40000,50000,60000,68335]) #虚假的x值，用来等间距分割
x_index = ['1690','3000','4000','5000','6000','7000','8000','9000','10000','20000','30000','40000','50000','60000','68335']
method=['pso']
labelname=['PSO'] #八个'2-wise','Filtrerad 2-wise'
#mark=['^','<','>','X','P']
color=['green']#,'olivedrab','orangered','darkorange','goldenrod','royalblue','darkblue']

#ODD覆盖率
plt.figure(dpi=600,figsize=(13,15))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.3)

plt.subplot(311)
for k in range(225):
    if k ==53:
        y=data[data['方法']=="pso"].场景库覆盖率[15*k:15*k+15]
        plt.plot(x, y, color='red',marker='s', label='PSO',linewidth=3,markersize=2)
    if k !=53:
        y=data[data['方法']=="pso"].场景库覆盖率[15*k:15*k+15]
        plt.plot(x, y, color=color[0],marker='s', label='PSO',linewidth=0.3,markersize=2)
        
plt.xticks(x,x_index,rotation=90,fontsize=11)
plt.yticks(fontsize=12)
plt.xlabel('Number of Scenarios Tested',fontsize=20) #X轴标签
plt.ylabel("Input coverage",fontsize=20) #Y轴标签

plt.subplot(312)
for k in range(225): 
    if k ==98:
        y=data[data['方法']=="pso"].碰撞场景个数[15*k:15*k+15]
        plt.plot(x, y, color='red',marker='s', label='PSO',linewidth=3,markersize=2)
    if k !=98:
        y=data[data['方法']=="pso"].碰撞场景个数[15*k:15*k+15]
        plt.plot(x, y, color=color[0],marker='s', label='PSO',linewidth=0.3,markersize=2)

plt.xticks(x,x_index,rotation=90,fontsize=11)
plt.yticks(fontsize=12)
plt.xlabel('Number of Scenarios Tested',fontsize=20) #X轴标签
plt.ylabel("Number of Collsion Scenarios",fontsize=20) #Y轴标签
    

plt.subplot(313)
for k in range(225):
    if k ==53:
        y=data[data['方法']=="pso"].事故率估计误差[15*k:15*k+15]
        plt.plot(x, y, color='red',marker='s', label='PSO',linewidth=3,markersize=2)
    if k !=53:
        y=data[data['方法']=="pso"].事故率估计误差[15*k:15*k+15]
        plt.plot(x, y, color=color[0],marker='s', label='PSO',linewidth=0.3,markersize=2)
    
plt.xticks(x,x_index,rotation=90,fontsize=11)
plt.yticks(fontsize=12)
plt.xlabel('Number of Scenarios Tested',fontsize=20) #X轴标签
plt.ylabel("Estimation Error",fontsize=20) #Y轴标签